In [1]:
import pandas as pd
import os

# 改數字部分 填入聯盟ERA,FIP,聯盟總得分(總失分)和聯盟總投球局數

In [2]:
lg_ERA = 3.700  #填入該年度ERA
lg_FIP = 3.700  #填入該年度FIP

RS = 2512 #聯盟總得分
Inn = 5341.667 #聯盟總局數

# 一.係數處理(單一球季RE24)

In [3]:
excel_file_path = 'C:/Users/ASUS/OneDrive/Documents/實習/Rebas/CPBL_2023.xlsx'  # 請替換成你的 Excel 檔案路徑
sheet_name1 = 'AWAY-打席資訊'  # 請替換成你要載入的工作表名稱
sheet_name2 = 'HOME-打席資訊'

In [4]:
away = pd.read_excel(excel_file_path, sheet_name=sheet_name1)
home = pd.read_excel(excel_file_path, sheet_name=sheet_name2)

In [5]:
cols = ['開始出局數','結束出局數','打席結果','落點1','落點2','彈道','RE','RE24']

In [6]:
away_play = away[cols]
home_play = home[cols]

In [7]:
summary = pd.concat([home_play,away_play], ignore_index=True)

In [8]:
summary['出局數']=summary['結束出局數']-summary['開始出局數']

In [9]:
index_to_change = summary[(summary['出局數'] == 2) | (summary['出局數'] == 3)].index

# 將出局數為2的資料改成1
summary.loc[index_to_change, '出局數'] = 1

In [10]:
summary1 = summary.groupby("彈道").agg({"RE": "mean", "RE24": "mean"})
summary1

,RE,RE24
彈道,,
F,0.543461,0.048510
G,0.581828,-0.077254
L,0.557324,0.307350
P,0.539672,-0.300934


In [11]:
summary2 = summary.groupby("打席結果").agg({"RE": "mean", "RE24": "mean"})
summary2

,RE,RE24
打席結果,,
1B,0.570149,0.489063
2B,0.570234,0.804999
3B,0.551902,1.072049
E,0.461618,0.391954
FC,0.853660,-0.300385
FO,0.508323,-0.299837
GIDP,0.996253,-0.897900
GO,0.449233,-0.248814
HBP,0.596673,0.372947


In [47]:
summary2['RE24']['SO']

-0.3046655457837558

In [13]:
def classify_fly_ball(row):
    #if row['彈道'] == 'F' and row['落點1'] in ['7.0','8.0','9.0']:
        #return '外野飛球'
    if row['彈道'] == 'P'and row['落點1'] not in ['7.0','8.0','9.0']:
        return '內野飛球'
    #elif row['彈道'] == 'P' and row['落點1'] in ['1.0','2.0','3.0','4.0','5.0','6.0']:
        #return '內野飛球'
    elif row['彈道'] == 'F'and row['落點1'] not in ['1.0','2.0','3.0','4.0','5.0','6.0']:
        return '外野飛球'
    elif row['彈道'] == 'L':
        return '平飛球'
    elif row['彈道'] == 'G':
        return '滾地球'
    else:
        return '其他'

summary['飛球分類'] = summary.apply(classify_fly_ball, axis=1)

# 打印结果
summary

,開始出局數,結束出局數,打席結果,落點1,落點2,彈道,RE,RE24,出局數,飛球分類
0,0,1,FO,8.0,89D,F,0.540,-0.254,1,外野飛球
1,1,2,FO,7.0,7D,F,0.286,-0.176,1,外野飛球
2,2,3,GO,4.0,4,G,0.110,-0.110,1,滾地球
3,0,0,uBB,NaN,NaN,NaN,0.540,0.394,0,其他
4,0,1,FO,8.0,89XD,F,0.934,-0.197,1,外野飛球
...,...,...,...,...,...,...,...,...,...,...
23066,1,2,FO,7.0,7,F,0.294,-0.178,1,外野飛球
23067,2,3,FO,7.0,7,F,0.115,-0.115,1,外野飛球
23068,0,1,FO,5.0,5,L,0.555,-0.261,1,平飛球
23069,1,2,GO,3.0,3,G,0.294,-0.178,1,滾地球


In [14]:
summary3 = summary.groupby(["飛球分類"]).agg({"RE": "mean", "RE24": "mean"})

# 重新命名列名
#hitter_summary.rename(columns={"li值": "平均LI值", "WPA": "總WPA","打序": "PA"}, inplace=True)

# 輸出結果
summary3

,RE,RE24
飛球分類,,
內野飛球,0.539672,-0.300934
其他,0.531403,-0.069363
外野飛球,0.543461,0.048510
平飛球,0.557324,0.307350
滾地球,0.581828,-0.077254


In [15]:
dfs = {}

# 定義你想分組的欄位
group_columns = ['飛球分類']

# 分組和聚合計算出局數的次數和總和
for name, group in summary.groupby(group_columns):
    df1 = group.groupby(group_columns).agg({'出局數':'count'})
    df2 = group.groupby(group_columns).agg({'出局數':'sum'})
    dfs[name] = pd.merge(df1, df2, on=group_columns)

# 合併所有的 DataFrame
result = pd.concat(dfs.values())

result

C:\Users\ASUS\AppData\Local\Temp\ipykernel_5596\4216358398.py:7: FutureWarning: In a future version of pandas, a length 1 tuple will be returned when iterating over a groupby with a grouper equal to a list of length 1. Don't supply a list with a single grouper to avoid this warning.
  for name, group in summary.groupby(group_columns):


,出局數_x,出局數_y
飛球分類,,
內野飛球,1154,1139
其他,6206,4008
外野飛球,5639,4013
平飛球,2218,644
滾地球,7854,5576


In [16]:
result = result.rename(columns={'出局數_x': '總數', '出局數_y': '出局數量'})
result

,總數,出局數量
飛球分類,,
內野飛球,1154,1139
其他,6206,4008
外野飛球,5639,4013
平飛球,2218,644
滾地球,7854,5576


In [17]:
result['出局比例']=result['出局數量']/result['總數']

In [18]:
result

,總數,出局數量,出局比例
飛球分類,,,
內野飛球,1154,1139,0.987002
其他,6206,4008,0.645827
外野飛球,5639,4013,0.711651
平飛球,2218,644,0.290352
滾地球,7854,5576,0.709957


# 二.計算各投手[HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA]

In [19]:
cols = ['客場','主場','投手','打席結果','落點1','落點2','彈道']

In [20]:
away_play = away[cols]
home_play = home[cols]

In [21]:
away_play = away_play.rename(columns={'客場': '對手', '主場': '球隊'})
home_play = home_play.rename(columns={'客場': '球隊', '主場': '對手'})

In [22]:
away_play

,對手,球隊,投手,打席結果,落點1,落點2,彈道
0,富邦悍將,統一7-ELEVEn獅,江承諺,FO,8.0,89,F
1,富邦悍將,統一7-ELEVEn獅,江承諺,1B,8.0,8S,L
2,富邦悍將,統一7-ELEVEn獅,江承諺,1B,7.0,7S,G
3,富邦悍將,統一7-ELEVEn獅,江承諺,FO,8.0,89D,F
4,富邦悍將,統一7-ELEVEn獅,江承諺,GO,4.0,4,G
...,...,...,...,...,...,...,...
11752,樂天桃猿,中信兄弟,江忠城,FO,7.0,7,F
11753,樂天桃猿,中信兄弟,江忠城,FO,7.0,7,F
11754,樂天桃猿,中信兄弟,江忠城,FO,5.0,5,L
11755,樂天桃猿,中信兄弟,江忠城,GO,3.0,3,G


In [23]:
summary = pd.concat([home_play,away_play], ignore_index=True)
summary

,球隊,對手,投手,打席結果,落點1,落點2,彈道
0,富邦悍將,統一7-ELEVEn獅,陳仕朋,FO,8.0,89D,F
1,富邦悍將,統一7-ELEVEn獅,陳仕朋,FO,7.0,7D,F
2,富邦悍將,統一7-ELEVEn獅,陳仕朋,GO,4.0,4,G
3,富邦悍將,統一7-ELEVEn獅,陳仕朋,uBB,NaN,NaN,NaN
4,富邦悍將,統一7-ELEVEn獅,陳仕朋,FO,8.0,89XD,F
...,...,...,...,...,...,...,...
23066,中信兄弟,樂天桃猿,江忠城,FO,7.0,7,F
23067,中信兄弟,樂天桃猿,江忠城,FO,7.0,7,F
23068,中信兄弟,樂天桃猿,江忠城,FO,5.0,5,L
23069,中信兄弟,樂天桃猿,江忠城,GO,3.0,3,G


#### 1. 計算三振,觸身球,保送和全壘打數量

In [24]:
def calculate_so_go_count(series):
    so_count = (series == 'SO').sum()
    ubb_count = (series == 'uBB').sum()
    hbp_count = (series == 'HBP').sum()
    hr_count = (series == 'HR').sum()
    return pd.Series({'SO': so_count,
                      'HR':hr_count
                     , 'uBB':ubb_count
                     , 'HBP':hbp_count})

In [25]:
resault_away=away_play.groupby(['投手','球隊'])['打席結果'].apply(calculate_so_go_count).reset_index()
resault_away = resault_away.rename(columns={'球隊':'球隊','level_2': '打席結果', '打席結果': '次數'})

resault_home=home_play.groupby(['投手','球隊'])['打席結果'].apply(calculate_so_go_count).reset_index()
resault_home = resault_home.rename(columns={'球隊':'球隊','level_2':'打席結果', '打席結果':'次數'})

merged = pd.merge(resault_away, resault_home, on=['投手', '球隊','打席結果'], how='outer')
merged.fillna(0, inplace=True)
merged['次數']=merged['次數_x']+merged['次數_y']

merged

,投手,球隊,打席結果,次數_x,次數_y,次數
0,伍茲,富邦悍將,SO,2.0,1.0,3.0
1,伍茲,富邦悍將,HR,0.0,0.0,0.0
2,伍茲,富邦悍將,uBB,9.0,5.0,14.0
3,伍茲,富邦悍將,HBP,1.0,0.0,1.0
4,伍鐸,味全龍,SO,12.0,8.0,20.0
...,...,...,...,...,...,...
583,陳聖文,富邦悍將,HBP,0.0,1.0,1.0
584,黃子宸,富邦悍將,SO,0.0,0.0,0.0
585,黃子宸,富邦悍將,HR,0.0,0.0,0.0
586,黃子宸,富邦悍將,uBB,0.0,0.0,0.0


In [26]:
playresult_items = merged['投手'].unique()
playresult_items

array(['伍茲', '伍鐸', '余德龍', '余謙', '傅于剛', '克維斯', '劉予承', '劉家愷', '劉軒荅', '勝騎士',
       '古林睿煬', '吳世豪', '吳俊偉', '吳俊杰', '吳君奕', '吳哲源', '呂偉晟', '呂彥青', '呂詠臻',
       '姚杰宏', '威能帝', '威達爾', '安得勝', '官大元', '富藍戈', '岳少華', '布萊威', '布里悍',
       '布雷克', '廖乙忠', '廖任磊', '張景淯', '張梓軒', '張祖恩', '彭識穎', '徐基麟', '徐若熙',
       '德保拉', '恩力\t', '施子謙', '曹祐齊', '曾仁和', '曾家輝', '曾峻岳', '朱俊祥', '李其峰',
       '李吳永勤', '李子強', '李建勲', '李振昌', '李超', '林凱威', '林子崴', '林子昱', '林華偉',
       '林詔恩', '林逸達', '森榮鴻', '楊孟沅', '楊彬', '楊志龍', '歐書誠', '江國謙', '江國豪',
       '江少慶', '江忠城', '江承峰', '江承諺', '泰樂', '游霆崴', '潘威倫', '猛威爾', '王凱程',
       '王奕凱', '王尉永', '王志煊', '王溢正', '王維中', '王躍霖', '王鏡銘', '瑞恩', '由規', '盧孟揚',
       '羅力', '羅國華', '羅昂', '羅華韋', '肯特', '胡智爲', '舒治浩', '艾士特', '艾普勒', '艾里歐',
       '范柏絜', '莊昕諺', '莊玉彬', '蔡齊哲', '藍愷青', '蘇俊璋', '蘭道爾', '許峻暘', '謝榮豪',
       '豪勁', '賴智垣', '賴知頎', '賴鴻誠', '趙璟榮', '道博格', '邱浩鈞', '邱駿威', '郭俊麟',
       '郭郁政', '鄭凱文', '鄭浩均', '鄭鈞仁', '金安', '鋼龍', '錡龍', '陳仕朋', '陳克羿', '陳冠偉',
       '陳冠勳', '陳冠宇', '陳柏豪', '陳琥', '陳禹勳', '陳韋霖', '陳韻文', '陳鴻文', '雷法\t',
    

In [27]:
merged['次數'] = merged['次數'].astype(int)

merged = pd.pivot_table(merged, index=['投手', '球隊'], columns=['打席結果'], values=['次數'], aggfunc='sum', fill_value=0)

merged

次數            
打席結果            HBP  HR  SO uBB
投手  球隊                         
伍茲  富邦悍將          1   0   3  14
伍鐸  味全龍           5   3  20  27
余德龍 樂天桃猿          0   0   0   0
余謙  中信兄弟          4   7  38  24
傅于剛 統一7-ELEVEn獅   0   1   1   2
...              ..  ..  ..  ..
黃偉晟 樂天桃猿          3   0  14  12
黃子宸 富邦悍將          0   0   0   0
黃子鵬 樂天桃猿          9  12  72  44
黃弘毅 中信兄弟          0   0   0   1
黃竣彥 統一7-ELEVEn獅   1   1  12  11

[147 rows x 4 columns]

#### 2.整理飛球分類 並計算數量

In [28]:
def classify_fly_ball(row):
    #if row['彈道'] == 'F' and row['落點1'] in ['7.0','8.0','9.0']:
        #return '外野飛球'
    if row['彈道'] == 'P' and row['落點1'] not in ['7.0','8.0','9.0']:
        return '內野飛球'
    #elif row['彈道'] == 'P' and row['落點1'] in ['1.0','2.0','3.0','4.0','5.0','6.0']:
        #return '內野飛球'
    elif row['彈道'] == 'F' and row['落點1'] not in ['1.0','2.0','3.0','4.0','5.0','6.0']:
        return '外野飛球'
    elif row['彈道'] == 'L':
        return '平飛球'
    elif row['彈道'] == 'G':
        return '滾地球'
    else:
        return '其他'

summary['飛球分類'] = summary.apply(classify_fly_ball, axis=1)

# 打印结果
summary


,球隊,對手,投手,打席結果,落點1,落點2,彈道,飛球分類
0,富邦悍將,統一7-ELEVEn獅,陳仕朋,FO,8.0,89D,F,外野飛球
1,富邦悍將,統一7-ELEVEn獅,陳仕朋,FO,7.0,7D,F,外野飛球
2,富邦悍將,統一7-ELEVEn獅,陳仕朋,GO,4.0,4,G,滾地球
3,富邦悍將,統一7-ELEVEn獅,陳仕朋,uBB,NaN,NaN,NaN,其他
4,富邦悍將,統一7-ELEVEn獅,陳仕朋,FO,8.0,89XD,F,外野飛球
...,...,...,...,...,...,...,...,...
23066,中信兄弟,樂天桃猿,江忠城,FO,7.0,7,F,外野飛球
23067,中信兄弟,樂天桃猿,江忠城,FO,7.0,7,F,外野飛球
23068,中信兄弟,樂天桃猿,江忠城,FO,5.0,5,L,平飛球
23069,中信兄弟,樂天桃猿,江忠城,GO,3.0,3,G,滾地球


In [29]:
def calculate_so_go_count(series):
    iff_count = (series == '內野飛球').sum()
    off_count = (series == '外野飛球').sum()
    ld_count = (series == '平飛球').sum()
    gr_count = (series == '滾地球').sum()
    return pd.Series({'內野飛球': iff_count,
                      '外野飛球':off_count
                     , '平飛球':ld_count
                     , '滾地球':gr_count})

In [30]:
resault2=summary.groupby(['投手','球隊'])['飛球分類'].apply(calculate_so_go_count).reset_index()
resault2 = resault2.rename(columns={'level_2': '擊球分類', '飛球分類': '次數'})

resault2['次數'] = resault2['次數'].astype(int)

final_resault2 = pd.pivot_table(resault2, index=['投手','球隊'], columns='擊球分類', values='次數', aggfunc='sum', fill_value=0)

final_resault2

,擊球分類,內野飛球,外野飛球,平飛球,滾地球
投手,球隊,,,,
伍茲,富邦悍將,2,9,3,22
伍鐸,味全龍,12,98,39,130
余德龍,樂天桃猿,0,2,0,2
余謙,中信兄弟,13,62,27,109
傅于剛,統一7-ELEVEn獅,1,5,2,8
...,...,...,...,...,...
黃偉晟,樂天桃猿,6,35,6,57
黃子宸,富邦悍將,1,0,0,5
黃子鵬,樂天桃猿,46,163,66,250


In [31]:
pitcher = pd.merge(merged, final_resault2,on=['投手','球隊'])
pitcher.rename(columns={('次數', 'HBP'): 'HBP',
                   ('次數', 'HR'): 'HR',
                   ('次數', 'SO'): 'SO',
                   ('次數', 'uBB'): 'uBB'}, inplace=True)
pitcher

C:\Users\ASUS\AppData\Local\Temp\ipykernel_5596\2221179295.py:1: FutureWarning: merging between different levels is deprecated and will be removed in a future version. (2 levels on the left, 1 on the right)
  pitcher = pd.merge(merged, final_resault2,on=['投手','球隊'])


,,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球
投手,球隊,,,,,,,,
伍茲,富邦悍將,1,0,3,14,2,9,3,22
伍鐸,味全龍,5,3,20,27,12,98,39,130
余德龍,樂天桃猿,0,0,0,0,0,2,0,2
余謙,中信兄弟,4,7,38,24,13,62,27,109
傅于剛,統一7-ELEVEn獅,0,1,1,2,1,5,2,8
...,...,...,...,...,...,...,...,...,...
黃偉晟,樂天桃猿,3,0,14,12,6,35,6,57
黃子宸,富邦悍將,0,0,0,0,1,0,0,5
黃子鵬,樂天桃猿,9,12,72,44,46,163,66,250


In [32]:
pitcher.reset_index(drop=False, inplace=True)
pitcher = pitcher[['投手', '球隊', 'HBP', 'HR', 'SO', 'uBB', '內野飛球', '外野飛球', '平飛球', '滾地球']]
pitcher

,投手,球隊,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球
0,伍茲,富邦悍將,1,0,3,14,2,9,3,22
1,伍鐸,味全龍,5,3,20,27,12,98,39,130
2,余德龍,樂天桃猿,0,0,0,0,0,2,0,2
3,余謙,中信兄弟,4,7,38,24,13,62,27,109
4,傅于剛,統一7-ELEVEn獅,0,1,1,2,1,5,2,8
...,...,...,...,...,...,...,...,...,...,...
142,黃偉晟,樂天桃猿,3,0,14,12,6,35,6,57
143,黃子宸,富邦悍將,0,0,0,0,1,0,0,5
144,黃子鵬,樂天桃猿,9,12,72,44,46,163,66,250
145,黃弘毅,中信兄弟,0,0,0,1,0,1,0,4


#### 3.計算聯盟定數 
# (聯盟ERA和FIP要自填)

In [33]:
uBB = pitcher['uBB'].sum()
HBP = pitcher['HBP'].sum()
SO = pitcher['SO'].sum()
HR = pitcher['HR'].sum()
滾地球 = pitcher['滾地球'].sum()
IFF = pitcher['內野飛球'].sum()
OFF = pitcher['外野飛球'].sum()
平飛球 = pitcher['平飛球'].sum()


constant = (lg_ERA - (
    (summary2['RE24']['uBB'] * uBB 
     + summary2['RE24']['HBP'] * HBP 
     + summary2['RE24']['SO'] * SO 
     + summary2['RE24']['HR'] * HR 
     + summary3['RE24']['滾地球'] * 滾地球 
     + summary3['RE24']['內野飛球'] * IFF
     + summary3['RE24']['外野飛球'] * OFF 
     + summary3['RE24']['平飛球'] * 平飛球)) /
    (SO + result['出局比例']['滾地球'] * 滾地球 + result['出局比例']['平飛球'] * 平飛球 + result['出局比例']['內野飛球'] * IFF + result['出局比例']['外野飛球'] * OFF) * 27
)
constant

3.432528481428009

In [34]:
lg_tRA = (summary2['RE24']['uBB'] * uBB 
     +summary2['RE24']['HBP'] * HBP 
     + summary2['RE24']['SO'] * SO 
     + summary2['RE24']['HR'] * HR 
     + summary3['RE24']['滾地球'] * 滾地球 
     +summary3['RE24']['內野飛球'] * IFF
     +summary3['RE24']['外野飛球'] * OFF 
     + summary3['RE24']['平飛球'] * 平飛球)/((SO + result['出局比例']['滾地球'] * 滾地球 + result['出局比例']['平飛球'] * 平飛球 + result['出局比例']['內野飛球'] * IFF + result['出局比例']['外野飛球'] * OFF) * 27)+constant
                
lg_tRA

3.432895383373924

In [35]:
pitcher['tRA']=((pitcher['uBB']*summary2['RE24']['uBB']
                 +pitcher['HBP']*summary2['RE24']['HBP']
                 +pitcher['SO']*summary2['RE24']['SO']
                 +pitcher['HR']*summary2['RE24']['HR']
                 +pitcher['滾地球']* summary3['RE24']['滾地球']
                 +pitcher['內野飛球']*summary3['RE24']['內野飛球']
                 +pitcher['外野飛球']*summary3['RE24']['外野飛球']
                 +pitcher['平飛球']*summary3['RE24']['平飛球'])/
                (pitcher['SO']
                 +pitcher['滾地球']*result['出局比例']['滾地球']
                +pitcher['平飛球']*result['出局比例']['平飛球']
                +pitcher['內野飛球']*result['出局比例']['內野飛球']
                +pitcher['外野飛球']*result['出局比例']['外野飛球'])*27)+constant

In [36]:
pitcher

,投手,球隊,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA
0,伍茲,富邦悍將,1,0,3,14,2,9,3,22,6.798954
1,伍鐸,味全龍,5,3,20,27,12,98,39,130,5.216280
2,余德龍,樂天桃猿,0,0,0,0,0,2,0,2,2.886611
3,余謙,中信兄弟,4,7,38,24,13,62,27,109,4.836925
4,傅于剛,統一7-ELEVEn獅,0,1,1,2,1,5,2,8,8.129997
...,...,...,...,...,...,...,...,...,...,...,...
142,黃偉晟,樂天桃猿,3,0,14,12,6,35,6,57,2.947574
143,黃子宸,富邦悍將,0,0,0,0,1,0,0,5,-0.657257
144,黃子鵬,樂天桃猿,9,12,72,44,46,163,66,250,4.222592
145,黃弘毅,中信兄弟,0,0,0,1,0,1,0,4,4.144750


# 三.投手一般成績整理

In [37]:
sheet_name3 = 'AWAY-BOX-投球'  # 請替換成你要載入的工作表名稱
sheet_name4 = 'HOME-BOX-投球'

away1 = pd.read_excel(excel_file_path, sheet_name=sheet_name3)
home1 = pd.read_excel(excel_file_path, sheet_name=sheet_name4)

In [38]:
away1.rename(columns={'客場': '球隊'}, inplace=True)
home1.rename(columns={'主場': '球隊'}, inplace=True)

In [39]:
merged_stat = pd.concat([away1, home1], ignore_index=True)

#merged_stat = merged_stat.drop(["比賽編號","比賽日期","客場","主場","背號"], axis = 1)
merged_stat

,比賽編號,比賽日期,球場,球隊,主場,順序,背號,名稱,出局數,投球數,...,全壘打,保送,故意四壞,觸身,三振,暴投,投手犯規,失分,責失,客場
0,285,2023-10-25 18:35,嘉義市立棒球場,富邦悍將,統一7-ELEVEn獅,1,81,陳仕朋,15,71,...,1,1,0,0,2,0,0,2,1,NaN
1,285,2023-10-25 18:35,嘉義市立棒球場,富邦悍將,統一7-ELEVEn獅,2,17,黃子宸,6,19,...,0,0,0,0,0,0,0,0,0,NaN
2,285,2023-10-25 18:35,嘉義市立棒球場,富邦悍將,統一7-ELEVEn獅,3,50,吳世豪,1,13,...,0,1,0,0,0,0,0,2,1,NaN
3,285,2023-10-25 18:35,嘉義市立棒球場,富邦悍將,統一7-ELEVEn獅,4,30,陳韋霖,2,9,...,0,0,0,0,0,0,0,0,0,NaN
4,204,2023-10-24 18:35,樂天桃園棒球場,富邦悍將,樂天桃猿,1,12,江國豪,21,98,...,0,2,0,1,6,0,0,1,1,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2599,2,2023-04-02 17:05,臺中市洲際棒球場,中信兄弟,NaN,3,94,吳俊偉,2,15,...,0,0,0,0,1,0,0,0,0,味全龍
2600,2,2023-04-02 17:05,臺中市洲際棒球場,中信兄弟,NaN,4,97,徐基麟,6,20,...,0,0,0,0,0,0,0,0,0,味全龍
2601,1,2023-04-01 17:21,臺中市洲際棒球場,中信兄弟,NaN,1,46,德保拉,19,103,...,1,2,0,0,4,0,0,3,3,樂天桃猿
2602,1,2023-04-01 17:21,臺中市洲際棒球場,中信兄弟,NaN,2,50,蔡齊哲,2,8,...,0,0,0,0,0,0,0,0,0,樂天桃猿


In [40]:
# 此Park Factor為2023年
park_factors = {
    "臺北市立天母棒球場": 0.974586135,
    "臺南市立棒球場": 1.049876521,
    "新北市立新莊棒球場": 0.982791461,
    "樂天桃園棒球場": 1.096458186,
    "臺中市洲際棒球場":0.909723723,
    "嘉義市立棒球場":0.938006629,
    "斗六棒球場":1.054394354,
    "澄清湖棒球場":0.932267771,
    "花蓮縣立德興棒球場":1.012278069,
    "台東棒球村第一棒球場":0.726667302,
    "新竹市立棒球場":0.298210736
}

merged_stat["PF(tRA)"] = merged_stat["球場"].map(park_factors)

merged_stat

,比賽編號,比賽日期,球場,球隊,主場,順序,背號,名稱,出局數,投球數,...,保送,故意四壞,觸身,三振,暴投,投手犯規,失分,責失,客場,PF(tRA)
0,285,2023-10-25 18:35,嘉義市立棒球場,富邦悍將,統一7-ELEVEn獅,1,81,陳仕朋,15,71,...,1,0,0,2,0,0,2,1,NaN,0.938007
1,285,2023-10-25 18:35,嘉義市立棒球場,富邦悍將,統一7-ELEVEn獅,2,17,黃子宸,6,19,...,0,0,0,0,0,0,0,0,NaN,0.938007
2,285,2023-10-25 18:35,嘉義市立棒球場,富邦悍將,統一7-ELEVEn獅,3,50,吳世豪,1,13,...,1,0,0,0,0,0,2,1,NaN,0.938007
3,285,2023-10-25 18:35,嘉義市立棒球場,富邦悍將,統一7-ELEVEn獅,4,30,陳韋霖,2,9,...,0,0,0,0,0,0,0,0,NaN,0.938007
4,204,2023-10-24 18:35,樂天桃園棒球場,富邦悍將,樂天桃猿,1,12,江國豪,21,98,...,2,0,1,6,0,0,1,1,NaN,1.096458
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2599,2,2023-04-02 17:05,臺中市洲際棒球場,中信兄弟,NaN,3,94,吳俊偉,2,15,...,0,0,0,1,0,0,0,0,味全龍,0.909724
2600,2,2023-04-02 17:05,臺中市洲際棒球場,中信兄弟,NaN,4,97,徐基麟,6,20,...,0,0,0,0,0,0,0,0,味全龍,0.909724
2601,1,2023-04-01 17:21,臺中市洲際棒球場,中信兄弟,NaN,1,46,德保拉,19,103,...,2,0,0,4,0,0,3,3,樂天桃猿,0.909724
2602,1,2023-04-01 17:21,臺中市洲際棒球場,中信兄弟,NaN,2,50,蔡齊哲,2,8,...,0,0,0,0,0,0,0,0,樂天桃猿,0.909724


In [41]:
agg_dict = {
    "出局數": "sum",
    "面對打者": "sum",
    "安打": "sum",
    "全壘打": "sum",
    "保送": "sum",
    "故意四壞": "sum",
    "觸身": "sum",
    "三振": "sum",
    "暴投": "sum",
    "投手犯規": "sum",
    "失分": "sum",
    "責失": "sum",
    "PF(tRA)": "mean"
}
pitcher_total = merged_stat.groupby(["名稱",'球隊']).agg(agg_dict).reset_index()
pitcher_total

,名稱,球隊,出局數,面對打者,安打,全壘打,保送,故意四壞,觸身,三振,暴投,投手犯規,失分,責失,PF(tRA)
0,伍茲,富邦悍將,28,54,11,0,14,0,1,3,0,0,13,13,1.005153
1,伍鐸,味全龍,240,336,84,3,27,0,5,20,0,0,26,23,1.013163
2,余德龍,樂天桃猿,3,4,1,0,0,0,0,0,0,0,0,0,1.096458
3,余謙,中信兄弟,191,277,63,7,24,0,4,38,0,0,39,35,0.946187
4,傅于剛,統一7-ELEVEn獅,10,19,7,1,2,0,0,1,0,0,5,1,0.983266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,黃偉晟,樂天桃猿,94,133,26,0,12,0,3,14,0,0,8,4,1.035139
143,黃子宸,富邦悍將,6,6,1,0,0,0,0,0,0,0,0,0,0.938007
144,黃子鵬,樂天桃猿,436,650,174,12,44,0,9,72,0,0,64,58,1.015346
145,黃弘毅,中信兄弟,5,6,1,0,1,0,0,0,0,0,0,0,0.935137


In [42]:
pitcher_total["IP"] = pitcher_total["出局數"] / 3
pitcher_total["ERA"] = (pitcher_total["責失"] / pitcher_total["IP"]) * 9
pitcher_total["FIP"] = (13 *pitcher_total["全壘打"] + 3 * pitcher_total["保送"] + 2 * pitcher_total["觸身"] + 3 * pitcher_total["暴投"] - 2 * pitcher_total["三振"]) / pitcher_total["IP"] + 3.2
pitcher_total["K%"] = (pitcher_total["三振"] / pitcher_total["面對打者"]) * 100
pitcher_total["BB%"] = (pitcher_total["保送"] / pitcher_total["面對打者"]) * 100
pitcher_total["ERA+"] = (lg_ERA / pitcher_total["ERA"]) * 100
pitcher_total["FIP+"] = (lg_FIP / pitcher_total["FIP"]) * 100

# 移除指定的欄位
pitcher_total.drop(columns=["出局數","暴投","投手犯規","故意四壞","觸身"], inplace=True)
pitcher_total = pitcher_total.rename(columns={'名稱': '投手'})

# 列出計算結果
pitcher_total

,投手,球隊,面對打者,安打,全壘打,保送,三振,失分,責失,PF(tRA),IP,ERA,FIP,K%,BB%,ERA+,FIP+
0,伍茲,富邦悍將,54,11,0,14,3,13,13,1.005153,9.333333,12.535714,7.271429,5.555556,25.925926,29.515670,50.884086
1,伍鐸,味全龍,336,84,3,27,20,26,23,1.013163,80.000000,2.587500,4.325000,5.952381,8.035714,142.995169,85.549133
2,余德龍,樂天桃猿,4,1,0,0,0,0,0,1.096458,1.000000,0.000000,3.200000,0.000000,0.000000,inf,115.625000
3,余謙,中信兄弟,277,63,7,24,38,39,35,0.946187,63.666667,4.947644,4.692147,13.718412,8.664260,74.783069,78.855166
4,傅于剛,統一7-ELEVEn獅,19,7,1,2,1,5,1,0.983266,3.333333,2.700000,8.300000,5.263158,10.526316,137.037037,44.578313
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,黃偉晟,樂天桃猿,133,26,0,12,14,8,4,1.035139,31.333333,1.148936,3.646809,10.526316,9.022556,322.037037,101.458576
143,黃子宸,富邦悍將,6,1,0,0,0,0,0,0.938007,2.000000,0.000000,3.200000,0.000000,0.000000,inf,115.625000
144,黃子鵬,樂天桃猿,650,174,12,44,72,64,58,1.015346,145.333333,3.591743,4.314679,11.076923,6.769231,103.014049,85.753774
145,黃弘毅,中信兄弟,6,1,0,1,0,0,0,0.935137,1.666667,0.000000,5.000000,0.000000,16.666667,inf,74.000000


# 四.投手成績和tRA合併

## 這裡合併完可以檢查前面是否有算錯 EX:兩個檔案的保送或三振數是否一樣

In [43]:
all_D = pd.merge(pitcher_total, pitcher, on=['投手','球隊'], how='inner')
all_D

,投手,球隊,面對打者,安打,全壘打,保送,三振,失分,責失,PF(tRA),...,FIP+,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA
0,伍茲,富邦悍將,54,11,0,14,3,13,13,1.005153,...,50.884086,1,0,3,14,2,9,3,22,6.798954
1,伍鐸,味全龍,336,84,3,27,20,26,23,1.013163,...,85.549133,5,3,20,27,12,98,39,130,5.216280
2,余德龍,樂天桃猿,4,1,0,0,0,0,0,1.096458,...,115.625000,0,0,0,0,0,2,0,2,2.886611
3,余謙,中信兄弟,277,63,7,24,38,39,35,0.946187,...,78.855166,4,7,38,24,13,62,27,109,4.836925
4,傅于剛,統一7-ELEVEn獅,19,7,1,2,1,5,1,0.983266,...,44.578313,0,1,1,2,1,5,2,8,8.129997
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,黃偉晟,樂天桃猿,133,26,0,12,14,8,4,1.035139,...,101.458576,3,0,14,12,6,35,6,57,2.947574
143,黃子宸,富邦悍將,6,1,0,0,0,0,0,0.938007,...,115.625000,0,0,0,0,1,0,0,5,-0.657257
144,黃子鵬,樂天桃猿,650,174,12,44,72,64,58,1.015346,...,85.753774,9,12,72,44,46,163,66,250,4.222592
145,黃弘毅,中信兄弟,6,1,0,1,0,0,0,0.935137,...,74.000000,0,0,0,1,0,1,0,4,4.144750


In [44]:
pitcher = all_D[['投手', '球隊','IP','ERA','FIP', 'HBP', 'HR', 'SO', 'uBB', '內野飛球', '外野飛球', '平飛球', '滾地球','tRA','PF(tRA)']]
pitcher

,投手,球隊,IP,ERA,FIP,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA,PF(tRA)
0,伍茲,富邦悍將,9.333333,12.535714,7.271429,1,0,3,14,2,9,3,22,6.798954,1.005153
1,伍鐸,味全龍,80.000000,2.587500,4.325000,5,3,20,27,12,98,39,130,5.216280,1.013163
2,余德龍,樂天桃猿,1.000000,0.000000,3.200000,0,0,0,0,0,2,0,2,2.886611,1.096458
3,余謙,中信兄弟,63.666667,4.947644,4.692147,4,7,38,24,13,62,27,109,4.836925,0.946187
4,傅于剛,統一7-ELEVEn獅,3.333333,2.700000,8.300000,0,1,1,2,1,5,2,8,8.129997,0.983266
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,黃偉晟,樂天桃猿,31.333333,1.148936,3.646809,3,0,14,12,6,35,6,57,2.947574,1.035139
143,黃子宸,富邦悍將,2.000000,0.000000,3.200000,0,0,0,0,1,0,0,5,-0.657257,0.938007
144,黃子鵬,樂天桃猿,145.333333,3.591743,4.314679,9,12,72,44,46,163,66,250,4.222592,1.015346
145,黃弘毅,中信兄弟,1.666667,0.000000,5.000000,0,0,0,1,0,1,0,4,4.144750,0.935137


# 五.計算WAR
## RPW 也需額外手動輸入 做場次校正 
### 輸入方式 中職RPW = RPW(使用FG) x 120/162 

In [45]:
#RAR＝（1.356×聯盟平均tRA－PF補正後tRA）÷9×（局數）
pitcher["RAR"]=((1.365*lg_tRA-pitcher["tRA"]*pitcher["PF(tRA)"])/9*pitcher["IP"])

#RPW = 9*(MLB Runs Scored / MLB Innings Pitched)*1.5 + 3
RPW = 9*((RS/Inn)*1.5 +3) *120/162
pitcher["WAR(FG)"]=pitcher["RAR"]/RPW

#pitcher["WAR(1.02)"]=pitcher["RAR"]/6.8593172473
pitcher

C:\Users\ASUS\AppData\Local\Temp\ipykernel_5596\2451314533.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitcher["RAR"]=((1.365*lg_tRA-pitcher["tRA"]*pitcher["PF(tRA)"])/9*pitcher["IP"])
C:\Users\ASUS\AppData\Local\Temp\ipykernel_5596\2451314533.py:6: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  pitcher["WAR(FG)"]=pitcher["RAR"]/RPW


,投手,球隊,IP,ERA,FIP,HBP,HR,SO,uBB,內野飛球,外野飛球,平飛球,滾地球,tRA,PF(tRA),RAR,WAR(FG)
0,伍茲,富邦悍將,9.333333,12.535714,7.271429,1,0,3,14,2,9,3,22,6.798954,1.005153,-2.227646,-0.090178
1,伍鐸,味全龍,80.000000,2.587500,4.325000,5,3,20,27,12,98,39,130,5.216280,1.013163,-5.324793,-0.215556
2,余德龍,樂天桃猿,1.000000,0.000000,3.200000,0,0,0,0,0,2,0,2,2.886611,1.096458,0.168984,0.006841
3,余謙,中信兄弟,63.666667,4.947644,4.692147,4,7,38,24,13,62,27,109,4.836925,0.946187,0.772951,0.031290
4,傅于剛,統一7-ELEVEn獅,3.333333,2.700000,8.300000,0,1,1,2,1,5,2,8,8.129997,0.983266,-1.225203,-0.049598
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
142,黃偉晟,樂天桃猿,31.333333,1.148936,3.646809,3,0,14,12,6,35,6,57,2.947574,1.035139,5.691361,0.230395
143,黃子宸,富邦悍將,2.000000,0.000000,3.200000,0,0,0,0,1,0,0,5,-0.657257,0.938007,1.178314,0.047700
144,黃子鵬,樂天桃猿,145.333333,3.591743,4.314679,9,12,72,44,46,163,66,250,4.222592,1.015346,6.435211,0.260507
145,黃弘毅,中信兄弟,1.666667,0.000000,5.000000,0,0,0,1,0,1,0,4,4.144750,0.935137,0.149999,0.006072


In [46]:
#pitcher.to_excel('2023WAR_PF2.0.xlsx', index=True)